In [45]:
import os
import requests
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import Tool, initialize_agent
from langchain.agents.agent_types import AgentType
from unidecode import unidecode


In [46]:
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")

In [47]:
def get_location_by_ip():
    try:
        response = requests.get("http://ip-api.com/json/")
        data = response.json()
        lat = data.get("lat")
        lon = data.get("lon")
        city = unidecode(data.get("city", "Unknown"))
        return lat, lon, city
    except:
        return None, None, None


In [48]:
def get_weather(_: str) -> str:
    lat, lon, city = get_location_by_ip()
    if not lat or not lon:
        return "❌ Could not detect your location."

    url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={OPENWEATHER_API_KEY}&units=metric"
    res = requests.get(url)
    data = res.json()

    if "main" in data:
        weather = data["weather"][0]["description"]
        temp = data["main"]["temp"]
        humidity = data["main"]["humidity"]
        return f"📍 {city}\n🌤 {weather}\n🌡 {temp}°C\n💧 {humidity}%"
    else:
        return "⚠ Weather data not found."


In [49]:
weather_tool = Tool(
    name="get_weather",
    func=get_weather,
    description="Use this to get the current weather based on IP location"
)

In [50]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    google_api_key=GOOGLE_API_KEY
)

In [51]:
agent = initialize_agent(
    tools=[weather_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [52]:
query = "What's the weather like here?"
response = agent.run(query)
print("\n🌍 Final Weather Report:\n", response)



> Entering new AgentExecutor chain...
Thought: I need to get the current weather based on the user's IP location.
Action: get_weather
Action Input: ""
Observation: 📍 Ludhiana
🌤 clear sky
🌡 32.26°C
💧 79%
Thought:Thought: I now know the final answer
Final Answer: The weather in Ludhiana is clear sky with a temperature of 32.26°C and 79% humidity.

> Finished chain.

🌍 Final Weather Report:
 The weather in Ludhiana is clear sky with a temperature of 32.26°C and 79% humidity.
